In [2]:
print('hi')

hi


In [3]:
import numpy as np
import pandas as pd
import sys
import time

start = time.process_time()

np = [{'Case':"Normal",'aaa':"2018/041155-9-LE187",'asda':"Moba" ,'Estado':"1", 'Promedio':17},
      {'Case':"Normal",'aaa':"2018/041155-9-LE187",'asda':"Opko" ,'Estado':"No 1", 'Promedio':21},
      {'Case':"Normal",'aaa':"2018/041155-9-LE187",'asda':"BBB" ,'Estado':"No 1", 'Promedio':24} ,
     
      #Case1 : WinPrice<SECOND_PRICE -Typical case get the second highest after win
      {'Case':"Normal",'aaa':"2019/111051765-53-LE192",'asda':"Moba", 'Estado':"No 1", 'Promedio':74},
      {'Case':"Normal",'aaa':"2019/111051765-53-LE192",'asda':"Opko" ,'Estado':"1", 'Promedio':58},
      {'Case':"Normal",'aaa':"2019/111051765-53-LE192",'asda':"BBB", 'Estado':"No 1", 'Promedio':100},
   
      #Case 2:  WinPrice>SECOND_PRICE --> GAP TO SECOND = 0 --> Find the first lowest price after winning --> 420 (not 370)
      {'Case':"Not_Normal",'aaa':"2019/111058045-27-LQ192",'asda':"Opko", 'Estado':"No 1", 'Promedio':370},
      {'Case':"Not_Normal",'aaa':"2019/111058045-27-LQ192",'asda':"Moba" ,'Estado':"No 1", 'Promedio':425},
      {'Case':"Not_Normal",'aaa':"2019/111058045-27-LQ192",'asda':"Winpharm",'Estado':"No 1", 'Promedio':420},
     
      # Case 3: No Winner --> Take only Lowest_Price, and second_price (if there is) if second=win --> second
      {'Case':"Not_Normal",'aaa':"2019/111058088-25-LE1914",'asda':"Moba" ,'Estado':"1", 'Promedio':31},
      {'Case':"Not_Normal",'aaa':"2019/111058088-25-LE1914",'asda':"BBB" ,'Estado':"1", 'Promedio':40},
      {'Case':"Not_Normal",'aaa':"2019/111058088-25-LE1914",'asda':"Opko", 'Estado':"No 1", 'Promedio':26}]

